### COSINE SIMILARITY IN BIGQUERY 

In [1]:
PROJECT = 'qwiklabs-gcp-4a684069c4776675'
BUCKET = 'colaborative-filtering-agea'
REGION = 'us-central1'

In [47]:
USER_ID = 900
LAST_DATA_TIME = "DATETIME '2018-12-31'"
DAYS_RECOMMEND = 5

In [7]:
import google.datalab.bigquery as bq

Saque la query de aca ->
[Analyzing text semantic similarity ]('https://cloud.google.com/solutions/machine-learning/analyzing-text-semantic-similarity-using-tensorflow-and-cloud-dataflow#explore_similar_articles_in_bigquery')

In [48]:
def create_cosine_similarity_query(user_id, number_of_articles):
  embed_columns = ["v{}".format(i) for i in range(50)]
  embed_columns_str = ",".join(embed_columns)
  average_columns_str = ",".join(map(lambda x: 'AVG({})'.format(x),embed_columns))
  query = """
      SELECT
      c.articles_key as similar_article_id,
      SUM(vv1*vv2) / (SQRT(SUM(POW(vv1,2))) * SQRT(SUM(POW(vv2,2)))) AS similarity,
      d.title,
      d.body,
      d.section_1,
      d.tag_1
    FROM
      (SELECT
         input_embed.val input_embed, articles_embed.key articles_key, articles_embed.val articles_embeds

        FROM (
          SELECT
              [{}] val
          FROM
            `AGEA_ASL.Dataset_D`
          WHERE user_id = '{}'
        )  input_embed
        CROSS JOIN
        (
          SELECT
          content_id key,
          [{}] val
          FROM `AGEA_ASL.Dataset_C`
          WHERE DATETIME_DIFF({}, CAST(day AS DATETIME),DAY) < {}
          AND content_id not in (SELECT content_id FROM `AGEA_ASL.Dataset_B` WHERE user_id='{}')
        ) articles_embed
    ) c
    , UNNEST(c.input_embed) vv1 with offset ind1 JOIN UNNEST(c.articles_embeds) vv2 with offset ind2 ON (ind1=ind2)
    LEFT JOIN `AGEA_ASL.Dataset_A` AS d ON c.articles_key = d.content_id
    GROUP BY c.articles_key,d.title,d.body,d.section_1,d.tag_1
    ORDER BY similarity DESC
    LIMIT {}
  """.format(average_columns_str,user_id, embed_columns_str,LAST_DATA_TIME,DAYS_RECOMMEND,user_id,number_of_articles)
  return query
  

In [49]:
print(create_cosine_similarity_query(USER_ID,4))


      SELECT
      c.articles_key as similar_article_id,
      SUM(vv1*vv2) / (SQRT(SUM(POW(vv1,2))) * SQRT(SUM(POW(vv2,2)))) AS similarity,
      d.title,
      d.body,
      d.section_1,
      d.tag_1
    FROM
      (SELECT
         input_embed.val input_embed, articles_embed.key articles_key, articles_embed.val articles_embeds

        FROM (
          SELECT
              [AVG(v0),AVG(v1),AVG(v2),AVG(v3),AVG(v4),AVG(v5),AVG(v6),AVG(v7),AVG(v8),AVG(v9),AVG(v10),AVG(v11),AVG(v12),AVG(v13),AVG(v14),AVG(v15),AVG(v16),AVG(v17),AVG(v18),AVG(v19),AVG(v20),AVG(v21),AVG(v22),AVG(v23),AVG(v24),AVG(v25),AVG(v26),AVG(v27),AVG(v28),AVG(v29),AVG(v30),AVG(v31),AVG(v32),AVG(v33),AVG(v34),AVG(v35),AVG(v36),AVG(v37),AVG(v38),AVG(v39),AVG(v40),AVG(v41),AVG(v42),AVG(v43),AVG(v44),AVG(v45),AVG(v46),AVG(v47),AVG(v48),AVG(v49)] val
          FROM
            `AGEA_ASL.Dataset_D`
          WHERE user_id = '900'
        )  input_embed
        CROSS JOIN
        (
          SELECT
          content_id key,

In [50]:
def recommend_articles(user_id, number_of_articles):
  query = create_cosine_similarity_query(user_id, number_of_articles)
  return bq.Query(query).execute().result().to_dataframe()
  

In [55]:
def get_articles_readed_by_user(user_id, number_of_articles):
  query = """ 
    SELECT  
      a.title,
      a.body, 
      a.section_1,
      a.tag_1
    FROM `AGEA_ASL.Dataset_A` a 
    RIGHT JOIN `AGEA_ASL.Dataset_B` b ON a.content_id = b.content_id 
    WHERE b.user_id = '{}'
    LIMIT {}
  """.format(user_id, number_of_articles)
  return bq.Query(query).execute().result().to_dataframe()

### ARTICLES READED VS RECOMMENDED

In [56]:
articles_readed = get_articles_readed_by_user(USER_ID,100)
articles_readed.head(100)

,title,body,section_1,tag_1
0,India acaba de levantar la estatua más grande ...,la india ha dado por finalizada la construcció...,Mundo,India
1,Derrame en Vaca Muerta: hay 45 hectáreas afec...,el derrame de horas de crudo en vaca muerta...,Sociedad,Vaca Muerta
2,"Por primera vez en el país, reconocieron a dos...",por primera vez en argentina la partida de na...,Sociedad,Mendoza
3,"Marcelo Gallardo: ""Si fue un acto de indiscipl...",no hay otro tema en la agenda futbolera la pe...,Deportes - Fútbol - River Plate,Marcelo Gallardo
4,Salió el fallo de la Conmebol: desestima el re...,la espera se prolongó más de lo deseado pero ...,Deportes - Fútbol,River Plate
5,"El planeta, contrarreloj: dramático llamado de...",destacados científicos internacionales alertar...,Sociedad,Cambio Climático y Calentamiento Global
6,Los Leoncitos golearon a pibes que por primera...,los leoncitos que venían de ganarle a a ...,Deportes - Polideportivo,Juegos Olímpicos de la Juventud
7,Así creció en una semana la grieta de Diamante...,la barranca de diamante va cediendo la compar...,Sociedad,Medio Ambiente
8,El video del ataque de furia de Luis Miguel qu...,luis miguel confirmó esta semana su fama de se...,Espectáculos - Música,Luis Miguel
9,Copa Libertadores: Gremio reclamará el resulta...,lo que parecía ser una anécdota más de una noc...,Deportes - Fútbol,River Plate


In [57]:
recommended_articles = recommend_articles(USER_ID, 20)
recommended_articles.head(20)

,similar_article_id,similarity,title,body,section_1,tag_1
0,5XJfNA3Cm,0.800405,"Dólar hoy: a cuánto cotiza, banco por banco",el dólar abrió este jueves con una nueva alza ...,Economía - Economía,Dólar hoy
1,9Iia3ZDG0,0.777571,"Scaloni y la relación con Messi: ""Tenemos una ...",lionel scaloni el entrenador interino de la s...,Deportes - Selección Nacional,Lionel Scaloni
2,bYBbmVRgu,0.771428,River y una curiosa descripción de lo que fue ...,difícilmente river pueda tener en su historia ...,Deportes - Fútbol - River Plate,River Plate
3,fi397HXHv,0.761563,Marco Ruben se aleja de Rosario Central y se l...,los caminos de marco ruben y rosario central c...,Deportes - Fútbol - Rosario Central,Rosario Central
4,C2cAMeO-C,0.758170,Godoy Cruz anunció a su nuevo entrenador: otra...,godoy cruz de mendoza anunció la contratación ...,Deportes - Fútbol - Godoy Cruz,Godoy Cruz
5,TaIkn8ICt,0.756450,Corte total y protesta en la Ruta 2: desvían e...,hay un corte total en la ruta a la altura de...,Sociedad,Mar del Plata
6,tSP9MvgpN,0.738774,Maxi Rodríguez vuelve a Newell's y el club le ...,la fiera vuelve a casa con esa frase la dir...,Deportes - Fútbol - Newell´s Old Boys,superliga20182019
7,Gb-0Q6pK6,0.738227,"Con un fuerte comunicado, Toluca criticó al Mo...",el club toluca de méxico confirmó a través de ...,Deportes - Fútbol,Godoy Cruz
8,FFh7WONpp,0.736817,"Argentinos presentó a su nuevo DT, que tendrá ...",argentinos juniors presentó oficialmente a su ...,Deportes - Fútbol - Argentinos Juniors,Argentinos Juniors
9,P0MofmAlo,0.733417,Sigue el calor y se esperan lluvias: cómo esta...,aunque se esperan lluvias para el resto de est...,Ciudades,Servicio Meteorológico Nacional
